In [1]:
%matplotlib inline

---
**作者**:	Zbigniew Jędrzejewski-Szmek

---

这一章是关于Python语言的高级特性-从不是每种语言都有这些特性的角度来说，也可以从他们在更复杂的程序和库中更有用这个角度来说，但是，并不是说特别专业或特别复杂。

需要强调的是本章是纯粹关于语言本身-关于由特殊语法支持的特性，用于补充Python标准库的功能，聪明的外部模块不会实现这部分特性。

开发Python程序语言的流程、语法是惟一的因为非常透明，提议的修改会在公共邮件列表中从多种角度去评估，最终的决策是来自于想象中的用例的重要性、带来更多语言特性所产生的负担、与其他语法的一致性及提议的变化是否易于读写和理解的权衡。这个流程被定型在Python增强建议中-[PEPs](http://www.python.org/dev/peps/)。因此，本章中的特性都是在显示出确实解决了现实问题，并且他们的使用尽可能简洁后才被添加的。

## 2.1.1 迭代器、生成器表达式和生成器

### 2.1.1.1 迭代器

---
简洁

重复的工作是浪费，用一个标准的特性替代不同的自产方法通常使事物的可读性和共用性更好。
Guido van Rossum — [为Python添加可选的静态输入](http://www.artima.com/weblogs/viewpost.jsp?thread=86641)

---

迭代器是继承了[迭代协议](http://docs.python.org/dev/library/stdtypes.html#iterator-types)的对象-本质上，这意味着它有一个[next](http://docs.python.org/2.7/library/stdtypes.html#iterator.next)方法，调用时会返回序列中的下一个项目，当没有东西可返回时，抛出[StopIteration](http://docs.python.org/2.7/library/exceptions.html#exceptions.StopIteration)异常。

迭代器对象只允许循环一次。它保留了单次迭代中的状态（位置），或者从另外的角度来看，序列上的每次循环需要一个迭代器对象。这意味着我们可以在一个序列上同时循环多次。从序列上分离出循环逻辑使我们可以有不止一种方法去循环。

在容器上调用[\_\_iter\_\_](http://docs.python.org/2.7/reference/datamodel.html#object.__iter__)方法来创建一个迭代器对象是获得迭代器的最简单方式。[iter](http://docs.python.org/2.7/library/functions.html#iter)函数帮我们完成这项工作，节省了一些按键次数。

In [12]:
nums = [1,2,3]      # 注意 ... 变化: 这些是不同的对象
iter(nums)  

In [2]:
nums.__iter__()   

In [3]:
nums.__reversed__()  

In [4]:
it = iter(nums)
next(it)            # next(obj)是obj.next()的简便用法

1

In [5]:
it.next()

2

In [6]:
next(it)

3

In [7]:
next(it)

StopIteration: 

在一个循环上使用时，[StopIteration](http://docs.python.org/2.7/library/exceptions.html#exceptions.StopIteration) 被忍受了，使循环终止。但是，当显式调用时，我们可以看到一旦迭代器结束，再访问它会抛出异常。

使用[for..in](http://docs.python.org/2.7/reference/compound_stmts.html#for) 循环也使用`__iter__`方法。这个方法允许我们在序列上显式的开始一个循环。但是，如果我们已经有个迭代器，我们想要可以在一个循环中以同样的方式使用它。要做到这一点，迭代器及`next`也需要有一个称为`__iter__`的方法返回迭代器(`self`)。

Python中对迭代器的支持是普遍的：标准库中的所有的序列和无序容器都支持迭代器。这个概念也被扩展到其他的事情：例如文件对象支持按行循环。

In [10]:
f = open('./etc/fstab')
f is f.__iter__()

True

`文件`是迭代器本身，它的`__iter__`方法并不创建一个新的对象：仅允许一个单一线程的序列访问。

### 2.1.1.2 生成器表达式

创建迭代器对象的第二种方式是通过生成器表达式，这也是列表推导的基础。要增加明确性，生成器表达式通常必须被括号或表达式包围。如果使用圆括号，那么创建了一个生成器迭代器。如果使用方括号，那么过程被缩短了，我们得到了一个列表。

In [13]:
(i for i in nums)                    

<generator object <genexpr> at 0x105fbc320>

In [14]:
[i for i in nums]

[1, 2, 3]

In [15]:
list(i for i in nums)

[1, 2, 3]

在Python 2.7和3.x中，列表推导语法被扩展为**字典和集合推导**。当生成器表达式被大括号包围时创建一个`集合`。当生成器表达式包含一对`键:值`的形式时创建`字典`:

In [16]:
{i for i in range(3)}   

{0, 1, 2}

In [17]:
{i:i**2 for i in range(3)}   

{0: 0, 1: 1, 2: 4}

如果你还在前面一些Python版本，那么语法只是有一点不同：

In [18]:
set(i for i in 'abc')

{'a', 'b', 'c'}

In [19]:
dict((i, ord(i)) for i in 'abc')

{'a': 97, 'b': 98, 'c': 99}

生成器表达式非常简单，在这里没有什么多说的。只有一个疑难问题需要提及：在旧的Python中，索引变量（i）可以泄漏，在>=3以上的版本，这个问题被修正了。

### 2.1.1.3 生成器

---
生成器

生成器是一个可以产生一个结果序列而不是单一值的函数。

David Beazley — [协程和并发的有趣课程](http://www.dabeaz.com/coroutines/)

---

创建迭代器对应的第三种方法是调用生成器函数。**生成器**是包含关键字[yield](http://docs.python.org/2.7/reference/simple_stmts.html#yield)的函数。必须注意，只要这个关键词出现就会彻底改变函数的本质：这个`yield`关键字并不是必须激活或者甚至可到达，但是，会造成这个函数被标记为一个生成器。当普通函数被调用时，函数体内包含的指令就开始执行。当一个生成器被调用时，在函数体的第一条命令前停止执行。调用那个生成器函数创建一个生成器对象，继承迭代器协议。与调用普通函数一样，生成器也允许并发和递归。

当`next`被调用时，函数执行到第一个`yield`。每一次遇到`yield`语句都会给出`next`的一个返回值。执行完yield语句，就暂停函数的执行。

In [20]:
def f():
    yield 1
    yield 2
f()                                   

<generator object f at 0x105fbc460>

In [21]:
gen = f()
gen.next()

1

In [22]:
gen.next()

2

In [23]:
gen.next()

StopIteration: 

让我们进入一次调用生成器函数的生命周期。

In [24]:
def f():
    print("-- start --")
    yield 3
    print("-- middle --")
    yield 4
    print("-- finished --")
gen = f()
next(gen)

-- start --


3

In [25]:
next(gen)

-- middle --


4

In [26]:
next(gen)                      

-- finished --


StopIteration: 

与普通函数不同，当执行`f()`时会立即执行第一个`print`，函数赋值到`gen`没有执行函数体内的任何语句。只有当用`next`激活`gen.next()`时，截至到第一个yield的语句才会被执行。第二个`next`打印`-- middle --`，执行到第二个`yield`终止。 第三个`next`打印`-- finished --`，并且到达了函数末尾。因为没有找到`yield`，抛出异常。

当向调用者传递控制时，在yield之后函数内发生了什么？每一个生成器的状态被存储在生成器对象中。从生成器函数的角度，看起来几乎是在一个独立的线程运行，但是，这是一个假象：执行是非常严格的单线程，但是解释器记录并恢复`next`值请求间的状态。

为什么生成器有用？正如迭代器部分的提到的，生成器只是创建迭代对象的不同方式。用`yield`语句可以完成的所有事，也都可以用`next`方法完成。尽管如此，

Nevertheless, using a function and having the interpreter perform its magic to create an iterator has advantages. A function can be much shorter than the definition of a class with the required next and __iter__ methods. What is more important, it is easier for the author of the generator to understand the state which is kept in local variables, as opposed to instance attributes, which have to be used to pass data between consecutive invocations of next on an iterator object.

In [27]:
def f():
    print("-- start --")
    print("-- middle --")
    print("-- finished --")
gen = f()

-- start --
-- middle --
-- finished --
